In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import os

In [2]:
# # Define the dataset class

# class FashionDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform=None):
#         try:
#             self.data = pd.read_csv(csv_file)
#         except pd.errors.ParserError as e:
#             print(f"Error reading CSV file: {e}")

    #     self.root_dir = root_dir
    #     self.transform = transform

    #     # Add encoding for categorical variables (assuming masterCategory and baseColour are categorical)
    #     self.master_category_encoding = {category: idx for idx, category in enumerate(self.data['masterCategory'].unique())}
    #     self.color_encoding = {color: idx for idx, color in enumerate(self.data['baseColour'].unique())}

    # def __len__(self):
    #     return len(self.data)

    # def __getitem__(self, idx):
    #     img_name = str(self.data.iloc[idx, 0]) + '.jpg'
    #     img_path = os.path.join(self.root_dir, img_name)
    #     image = read_image(img_path)

    #     # Encode categorical variables
    #     master_category = torch.tensor(self.master_category_encoding[self.data.iloc[idx, 2]], dtype=torch.long)
    #     color_label = torch.tensor(self.color_encoding[self.data.iloc[idx, 5]], dtype=torch.long)

    #     if self.transform:
    #         image = self.transform(image)

    #     return {'image': image, 'master_category': master_category, 'color_label': color_label}
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import os

# Define the dataset class
class FashionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        try:
            self.data = pd.read_csv(csv_file,on_bad_lines='skip')
        except pd.errors.ParserError as e:
            print(f"Error reading CSV file: {e}")

        self.root_dir = root_dir
        self.transform = transform

        # Add encoding for categorical variables (assuming masterCategory and baseColour are categorical)
        self.master_category_encoding = {category: idx for idx, category in enumerate(self.data['masterCategory'].unique())}
        self.color_encoding = {color: idx for idx, color in enumerate(self.data['baseColour'].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = str(self.data.iloc[idx, 0]) + '.jpg'
        img_path = os.path.join(self.root_dir, img_name)
        image = read_image(img_path)

        # Encode categorical variables
        master_category = torch.tensor(self.master_category_encoding[self.data.iloc[idx, 2]], dtype=torch.long)
        color_label = torch.tensor(self.color_encoding[self.data.iloc[idx, 5]], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return {'image': image, 'master_category': master_category, 'color_label': color_label}

# Rest of your code...



In [3]:
class FashionModel(nn.Module):
    def __init__(self, num_master_categories, num_colors):
        super(FashionModel, self).__init__()
        self.conv_final = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()

        # Example input size (batch_size, channels, height, width)
        sample_image = torch.randn((3, num_channels, 512, 640))
        conv_out = self.conv_final(sample_image)
        flattened_size = self.flatten(conv_out).size(1)

        self.fc_master_category = nn.Linear(flattened_size, num_master_categories)
        self.fc_color = nn.Linear(flattened_size, num_colors)

    def forward(self, x):
        x = F.relu(self.conv_final(x))
        x = x.view(x.size(0), -1)
        master_category_output = self.fc_master_category(x)
        color_output = self.fc_color(x)
        return master_category_output, color_output

In [6]:
# Instantiate the dataset and dataloaders
from torchvision import transforms
csv_file = '/Users/akshararamprasad/Documents/Fashion/styles.csv'
root_dir = '/Users/akshararamprasad/Documents/Fashion/imagesdata'
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512,640)),
    transforms.ToTensor()
])
dataset = FashionDataset(csv_file, root_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
# Instantiate the model
num_channels = 3
num_master_categories = len(dataset.data['masterCategory'].unique())
num_colors = len(dataset.data['baseColour'].unique())
model = FashionModel(num_master_categories, num_colors)


In [7]:
# # Inference example
# model.eval()
# with torch.no_grad():
#     sample = next(iter(test_loader))
#     images = sample['image']
#     master_category_labels = sample['master_category']
#     color_labels = sample['color_label']

#     master_category_output, color_output = model(images)
#     predicted_master_category = torch.argmax(master_category_output, dim=1)
#     predicted_color = torch.argmax(color_output, dim=1)

#     print("Predicted Master Category:", predicted_master_category)
#     print("Predicted Color:", predicted_color)
# Inference example
model.eval()
with torch.no_grad():
    sample = next(iter(test_loader))
    images = sample['image']
    master_category_labels = sample['master_category']
    color_labels = sample['color_label']

    # Ensure images have 3 channels (adjust as needed)
    if images.size(1) == 1:
        images = torch.cat([images] * 3, dim=1)

    master_category_output, color_output = model(images)
    predicted_master_category = torch.argmax(master_category_output, dim=1)
    predicted_color = torch.argmax(color_output, dim=1)

    print("Predicted Master Category:", predicted_master_category)
    print("Predicted Color:", predicted_color)

# Evaluation
# ... (previous code)



Predicted Master Category: tensor([5])
Predicted Color: tensor([44])


In [8]:
master_category_encoding = {
    0: 'Apparel',
    1: 'Accessories',
    2: 'Footwear',
    3: 'Personal Care',
    4: 'Free Items',
    5: 'Sporting Goods',
    6: 'Home'
}

# Dictionary for baseColour
color_encoding = {
    0: 'Navy Blue',
    1: 'Blue',
    2: 'Silver',
    3: 'Black',
    4: 'Grey',
    5: 'Green',
    6: 'Purple',
    7: 'White',
    8: 'Beige',
    9: 'Brown',
    10: 'Bronze',
    11: 'Teal',
    12: 'Copper',
    13: 'Pink',
    14: 'Off White',
    15: 'Maroon',
    16: 'Red',
    17: 'Khaki',
    18: 'Orange',
    19: 'Yellow',
    20: 'Charcoal',
    21: 'Gold',
    22: 'Steel',
    23: 'Tan',
    24: 'Multi',
    25: 'Magenta',
    26: 'Lavender',
    27: 'Sea Green',
    28: 'Cream',
    29: 'Peach',
    30: 'Olive',
    31: 'Skin',
    32: 'Burgundy',
    33: 'Coffee Brown',
    34: 'Grey Melange',
    35: 'Rust',
    36: 'Rose',
    37: 'Lime Green',
    38: 'Mauve',
    39: 'Turquoise Blue',
    40: 'Metallic',
    41: 'Mustard',
    42: 'Taupe',
    43: 'Nude',
    44: 'Mushroom Brown',
    45: 'Fluorescent Green'
}
model.eval()
with torch.no_grad():
    sample = next(iter(test_loader))
    images = sample['image']

    master_category_output, color_output = model(images)
    predicted_master_category_index = torch.argmax(master_category_output, dim=1).item()
    predicted_color_index = torch.argmax(color_output, dim=1).item()

    # Get predicted labels from encoding dictionaries
    predicted_master_category_label = master_category_encoding[predicted_master_category_index]
    predicted_color_label = color_encoding[predicted_color_index]

    print("Predicted Master Category:", predicted_master_category_label)
    print("Predicted Color:", predicted_color_label)
# Save the color model
torch.save(model.fc_color.state_dict(), 'color_model.pth')

# Save the category model
torch.save(model.fc_master_category.state_dict(), 'category_model.pth')


Predicted Master Category: Sporting Goods
Predicted Color: Mushroom Brown
